In [ ]:
import os
import tensorflow as tf
import tensorflow.contrib.eager as tfe
from matplotlib import pyplot as plt
import numpy as np

tf.enable_eager_execution()

In [ ]:
cache_path = "./Cache/"
fname = 'bottle_neck.npz'
save_path = os.path.join(cache_path,fname)

device = "gpu:0" if tfe.num_gpus() else "cpu:0"

In [ ]:
class XceptionClassifier(tf.keras.Model):
    
    def __init__(self, n_classes):
        super(XceptionClassifier, self).__init__()
        # Define the layer(s) you would like to use for your classifier
    # 5 layers
#         self.dense_layer1 = tf.keras.layers.Dense(units=4096, activation='relu')
#         self.dense_layer2 = tf.keras.layers.Dense(units=2048, activation='relu')
#         self.dense_layer3 = tf.keras.layers.Dense(units=1024, activation='relu')
#         self.dense_layer4 = tf.keras.layers.Dense(units=256, activation='relu')
#         self.dense_layer5 = tf.keras.layers.Dense(units=n_classes)
#         # ... additional layers if you want...
    # 3 layers
#         self.dense_layer1 = tf.keras.layers.Dense(units=20, activation='relu')
#         self.dense_layer2 = tf.keras.layers.Dense(units=20, activation='relu')
#         self.dense_layer3 = tf.keras.layers.Dense(units=20, activation='relu')
#         self.dense_layer4 = tf.keras.layers.Dense(units=n_classes)
        
    # 2 layers
        self.dense_layer1 = tf.keras.layers.Dense(units=1024, activation='relu')
        self.dense_layer2 = tf.keras.layers.Dense(units=n_classes)
        
    def call(self, inputs):
        # Set this up appropriately, will depend on how many layers you choose
#         result = self.dense_layer1(inputs)
#         result = self.dense_layer2(result)
#         result = self.dense_layer3(result)
#         result = self.dense_layer4(result)
#         result = self.dense_layer5(result)
        
        # ... if additional layers...
#         result = self.dense_layer1(inputs)
#         result = self.dense_layer2(result)
#         result = self.dense_layer3(result)
#         result = self.dense_layer4(result)

        result = self.dense_layer1(inputs)
        result = self.dense_layer2(result)

        return result

In [ ]:
data = np.load(save_path)
train_bottle_necks, train_labels = data['bottle_necks'],  data['labels']
n_train_samples= len(train_bottle_necks)

In [ ]:
# Normalization if necessary
for i in range(n_train_samples):
    train_bottle_necks[i] = np.true_divide(train_bottle_necks[i], max(train_bottle_necks[i]))

In [ ]:
from sklearn.model_selection import train_test_split
X, X_test, Y, Y_test = train_test_split(train_bottle_necks,train_labels, test_size=0.2)
X_train, X_v, Y_train, Y_v = train_test_split(X,Y,test_size=0.2)

In [ ]:
batch_size = 20        # You will play around with this 
n_epochs = 200         # Choose num epochs based on when you think the parameters have converged
learning_rate = 0.001 # You will try different learning rates

train_loss_history = []
validation_loss_history = []

In [ ]:
train_images_dataset = tf.data.Dataset.from_tensor_slices(X_train)
train_labels_dataset = tf.data.Dataset.from_tensor_slices(Y_train)
train_dataset = tf.data.Dataset.zip((train_images_dataset, train_labels_dataset))
train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size)

In [ ]:
x_classifier = XceptionClassifier(n_classes=2)
optimizer = tf.train.AdamOptimizer(learning_rate) 

In [ ]:
with tf.device(device):
    losscount = 0
    for epoch in range(n_epochs):
        for batch, (images, labels) in enumerate(train_dataset):
            with tf.GradientTape() as tape:
                # Compute logits
                logits = x_classifier(images)
                xe_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))                
                train_loss_history.append(xe_loss.numpy())
            # Compute gradient and apply gradients
                                         
            grads = tape.gradient(xe_loss, x_classifier.variables)
            optimizer.apply_gradients(zip(grads, x_classifier.variables),global_step=tf.train.get_or_create_global_step())
            
            
            if batch % 10 == 0:
                print('\rEpoch: {}, Batch: {}, Loss: {}'.format(epoch, batch, train_loss_history[-1]), end='')
                #print('\rEpoch: {}, Batch: {}, Loss(validation): {}'.format(epoch, batch, validation_loss_history[-1]), end='')
                                         
        #for batch, (images,labels) in enumerate(validation_dataset):
        logits_v = x_classifier(X_v)
        xe_loss_v = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y_v, logits=logits_v))
        
        
        if len(validation_loss_history) >= 5:
            if xe_loss_v > np.array(validation_loss_history[-1:-3:-1]).mean():
                losscount +=1
            else:
                losscount = 0
                
        if losscount > 2:
            break
        validation_loss_history.append(xe_loss_v.numpy()) 
       
        print('\rEpoch: {}, Batch: {}, Train Loss: {},Count: {}'.format(epoch, batch, train_loss_history[-1],losscount))
        print('\rEpoch: {}, Batch: {}, Validation Loss: {},Count: {}'.format(epoch, batch, validation_loss_history[-1],losscount))
        

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(validation_loss_history)
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('loss', fontsize=14);

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(train_loss_history)
plt.xlabel('Iterations', fontsize=14)
plt.ylabel('loss', fontsize=14);

In [ ]:
logits = x_classifier(tf.constant(X_test))
y_pred = tf.nn.softmax(logits).numpy()

In [ ]:
accuracy_result_1=[]
for i in range(len(Y_test)):
    accuracy_result_1.append(Y_test[i] - np.argmax(y_pred,axis=1)[i])
accuracy_result_1.count(0)/len(Y_test)

In [ ]:
logits = x_classifier(tf.constant(X_train))
y_pred_train = tf.nn.softmax(logits).numpy()

In [ ]:
accuracy_result_2=[]
for i in range(len(Y_train)):
    accuracy_result_2.append(Y_train[i] - np.argmax(y_pred_train,axis=1)[i])
accuracy_result_2.count(0)/len(Y_train)